# DECORATORS PART 1: Theory


`Decorators`provide a very useful method to add functionality to existing functions and classes.   
Decorators are
functions that wrap other functions or classes.
One example for the use of decorator are static methods. Static methods could be function in the global scope
but are defined inside a class. There is no `self` and no reference to the instance.  



In [51]:
# level 1 @function
def hello(func):
    # Level 2 actual decorator
    def call_func(*args, **kwargs):
        # call func takes arguments of parents function and simply passes them through
            """Takes a arbitrary number of positional and keyword arguments."""
            print('Hello')
            # Call original function and return its result.
            return func(*args, **kwargs)
    return call_func
@hello
def add(a,b):
    return a+b

add(1,2)

# Class decorators 
It is strongly recommended that a function decorator always returns a function object and a class decorator always returns a class object.  
A function decorator should typically either return a function that returns the result of the call to the original function and do
something in addition or return the original function itself.

In [64]:
def mark(cls):
    cls.decorated_str = 'I am decorated.'
    return cls

@mark
class A(object):
    pass
A.decorated_str


'I am decorated.'

### Passing arguments to decorators 



In [60]:
# level 1 decorator arguments
def say(text):
    #level 2 input function
    def _say(func):
        #level 3 actual decorator
        def call_func(*args, **kwargs):
            print(text)
            return func(*args, **kwargs)
        return call_func
    return _say

@say("hello")
def add(a,b):
    return a+b

add(1,2)

hello


3

In [28]:
def info(arg1, arg2):
    print('Decorator arg1 = ' + str(arg1))
    print('Decorator arg2 = ' + str(arg2))
 
    def the_real_decorator(function):
 
        def wrapper(*args, **kwargs):
            print('Function {} args: {} kwargs: {}'.format(
                function.__name__, str(args), str(kwargs)))
            return function(*args, **kwargs)
 
        return wrapper
 
    return the_real_decorator
 
@info(3, 'Python')
def doubler(number):
    return number * 2
 
print(doubler(5))

Decorator arg1 = 3
Decorator arg2 = Python
Function doubler args: (5,) kwargs: {}
10


## Use case 1: static method of a class

In [29]:
class staticClass(object):
    @staticmethod
    def func():
        print("method used in static way")

In [30]:
a = staticClass
a.func()

method used in static way


##  Usecase 2: formatting HTML strings

In [19]:
def bold(func):
    def wrapper(*args):
        return "<b>" + func(*args) + "</b>"
    return wrapper
 
def italic(func):
    def wrapper(*args):
        return "<i>" + func(*args) + "</i>"
    return wrapper
 
@bold
@italic
def formatted_text(message):
    return "<begin>" +message + "<end>"
 
print(formatted_text("formatted text"))

<b><i><begin>formatted text<end></i></b>


# Usecase 3 : dynamically adding functionality


In [34]:
def decorator(cls):
    class Wrapper(cls):
        def doubler(self, value):
            return value * 2
    return Wrapper
 
@decorator
class MyActualClass:
    def __init__(self):
        print('in MyActualClass __init__()')
 
    def quad(self, value):
        return value * 4
obj = MyActualClass()
print(obj.quad(4))
print(obj.doubler(5))

in MyActualClass __init__()
16
10


# Usecase 4 argument checking 

In [69]:
import functools

# first layer decorator arguments
def check(*argtypes):
    """Function argument type checker.
    """
    # second layer return function
    def _check(func):
        """Takes the function.
        """
        @functools.wraps(func)
        # third layer actual decorator
        def __check(*args):
            """Takes the arguments
            """
            if len(args) != len(argtypes):
                msg = 'Expected %d but got %d arguments' % (len(argtypes),len(args))
                raise TypeError(msg)
            for arg, argtype in zip(args, argtypes):
                if not isinstance(arg, argtype):
                    msg = 'Expected %s but got %s' % (argtypes, tuple(type(arg) for arg in args))
                    raise TypeError(msg)
            return func(*args)
        return __check
    return _check
@check(int, int)
def add(x, y):
    """Add two integers."""
    return x + y

In [71]:
add("pi",2)

TypeError: Expected (<class 'int'>, <class 'int'>) but got (<class 'str'>, <class 'int'>)

# Usecase 5 CACHING

In [81]:
import functools
import pickle
def cached(func):
    """Decorator that caches."""
    cache = {}
    @functools.wraps(func)
    def _cached(*args, **kwargs):
        """Takes the arguments."""
        # dicts cannot be use as dict keys
        # dumps are strings and can be used
        key = pickle.dumps((args, kwargs))
        print(key)
        if key not in cache:
             cache[key] = func(*args, **kwargs)
        return cache[key]
    return _cached

Only the first call will print `calc`. All subsequent calls get the value from cache without newly calculating it:

In [83]:
@cached
def calculate_cached(a,b):
    print("calculated")
    return(a*b)

In [93]:
calculate_cached(3,5)
# 2Nd time value is cached
calculate_cached(3,5)


b'\x80\x03K\x03K\x05\x86q\x00}q\x01\x86q\x02.'
b'\x80\x03K\x03K\x05\x86q\x00}q\x01\x86q\x02.'


15